In [15]:
# !pip install -r ./requirements.txt

In [15]:
# !pip uninstall virny -y

In [4]:
# Install using an HTTP link
# !pip install git+https://github.com/DataResponsibly/Virny.git@feature/prepare_experiments_for_inprocessors

# Install using an SSH link
# !pip install git+ssh://git@github.com/DataResponsibly/Virny.git@feature/prepare_experiments_for_inprocessors

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

In [3]:
cur_folder_name = os.getcwd().split('/')[-1]
if cur_folder_name != "fairness-variance":
    os.chdir("../../../..")

print('Current location: ', os.getcwd())

Current location:  /home/dh3553/projects/fairness-variance


## Import dependencies

In [4]:
import copy

from virny.utils.custom_initializers import create_config_obj
from virny.datasets import ACSPublicCoverageDataset

from configs.constants import TEST_SET_FRACTION, EXPERIMENT_SEEDS

from source.experiment_interface import run_exp_iter_with_inprocessor

pip install 'aif360[LawSchoolGPA]'


## Define Input Variables

In [5]:
ROOT_DIR = os.getcwd()
EXPERIMENT_NAME = 'EGR_acs_pubcov'
DB_COLLECTION_NAME = 'one_repair_lvl_many_models'
FAIRNESS_INTERVENTION_NAME = 'EGR'
FAIR_INTERVENTION_PARAMS_LST = ['DemographicParity']
SAVE_RESULTS_DIR_PATH = os.path.join(ROOT_DIR, 'results', 'diff_fairness_interventions_exp',
                                     FAIRNESS_INTERVENTION_NAME, EXPERIMENT_NAME)

config_yaml_path = os.path.join(ROOT_DIR, 'notebooks', 'diff_fairness_interventions_exp',
                                FAIRNESS_INTERVENTION_NAME, EXPERIMENT_NAME, 'folk_CA_2018_config.yaml')
metrics_computation_config = create_config_obj(config_yaml_path=config_yaml_path)

## Define a db writer and custom fields to insert into your database

In [6]:
import os
from dotenv import load_dotenv

load_dotenv('./configs/secrets.env')
os.getenv("DB_NAME")

'fairness_variance'

In [7]:
from source.utils.db_functions import connect_to_mongodb

client, collection_obj, db_writer_func = connect_to_mongodb(DB_COLLECTION_NAME)

In [8]:
import uuid

custom_table_fields_dct = {
#     'session_uuid': str(uuid.uuid4()),
    'session_uuid': 'dffb2122-b870-40d5-92b2-f0f6bfb730f0',
}
print('Current session uuid: ', custom_table_fields_dct['session_uuid'])

Current session uuid:  dffb2122-b870-40d5-92b2-f0f6bfb730f0


## Initialize custom objects

In [9]:
data_loader = ACSPublicCoverageDataset(state=['CA'], year=2018, with_nulls=False,
                                       subsample_size=15_000, subsample_seed=42)
data_loader.X_data.head()

,SCHL,MAR,SEX,DIS,ESP,CIT,MIG,MIL,ANC,NATIVITY,DEAR,DEYE,DREM,ESR,ST,FER,RAC1P,AGEP,PINCP
0,19,5,1,2,0,1,3,4,1,1,2,2,2,6,6,0,1,21,3150.0
1,16,5,1,2,0,3,3,4,4,1,2,2,2,1,6,0,9,18,1600.0
2,13,5,2,2,1,1,1,0,2,1,2,2,2,6,6,2,1,16,0.0
3,20,1,2,2,0,4,1,4,1,2,2,2,2,6,6,2,8,43,0.0
4,16,1,2,2,0,4,1,4,1,2,2,2,2,6,6,0,6,54,0.0


In [10]:
data_loader.X_data.shape

(15000, 19)

## Run experiment iterations

### Experiment iteration 1

In [14]:
# Configs for an experiment iteration
exp_iter_num = 1
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'
exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage

In [ ]:
run_exp_iter_with_inprocessor(data_loader=exp_iter_data_loader,
                              experiment_seed=experiment_seed,
                              test_set_fraction=TEST_SET_FRACTION,
                              db_writer_func=db_writer_func,
                              fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                              metrics_computation_config=metrics_computation_config,
                              custom_table_fields_dct=custom_table_fields_dct,
                              dataset_name='ACSPublicCoverageDataset',
                              inprocessor_name='ExponentiatedGradientReduction',
                              verbose=True)

2024-01-09 20:30:44 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 100,
 'experiment_iteration': 'Exp_iter_1',
 'fair_intervention_params_lst': "['DemographicParity']",
 'model_init_seed': 100,
 'session_uuid': 'dffb2122-b870-40d5-92b2-f0f6bfb730f0'}




Multiple alphas:   0%|          | 0/1 [00:00<?, ?it/s]

2024-01-09 20:30:45 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed


intervention_option:  DemographicParity
cur_base_flow_dataset.X_train_val.columns:  Index(['cat__SCHL_1', 'cat__SCHL_10', 'cat__SCHL_11', 'cat__SCHL_12',
       'cat__SCHL_13', 'cat__SCHL_14', 'cat__SCHL_15', 'cat__SCHL_16',
       'cat__SCHL_17', 'cat__SCHL_18', 'cat__SCHL_19', 'cat__SCHL_2',
       'cat__SCHL_20', 'cat__SCHL_21', 'cat__SCHL_22', 'cat__SCHL_23',
       'cat__SCHL_24', 'cat__SCHL_3', 'cat__SCHL_4', 'cat__SCHL_5',
       'cat__SCHL_6', 'cat__SCHL_7', 'cat__SCHL_8', 'cat__SCHL_9',
       'cat__MAR_1', 'cat__MAR_2', 'cat__MAR_3', 'cat__MAR_4', 'cat__MAR_5',
       'cat__DIS_1', 'cat__DIS_2', 'cat__ESP_0', 'cat__ESP_1', 'cat__ESP_2',
       'cat__ESP_3', 'cat__ESP_4', 'cat__ESP_5', 'cat__ESP_6', 'cat__ESP_7',
       'cat__ESP_8', 'cat__CIT_1', 'cat__CIT_2', 'cat__CIT_3', 'cat__CIT_4',
       'cat__CIT_5', 'cat__MIG_1', 'cat__MIG_2', 'cat__MIG_3', 'cat__MIL_0',
       'cat__MIL_1', 'cat__MIL_2', 'cat__MIL_3', 'cat__MIL_4', 'cat__ANC_1',
       'cat__ANC_2', 'cat__ANC_3', 'c

Analyze multiple models:   0%|          | 0/1 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

### Experiment iteration 2

In [11]:
# Configs for an experiment iteration
exp_iter_num = 2
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'
exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage

In [12]:
run_exp_iter_with_inprocessor(data_loader=exp_iter_data_loader,
                              experiment_seed=experiment_seed,
                              test_set_fraction=TEST_SET_FRACTION,
                              db_writer_func=db_writer_func,
                              fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                              metrics_computation_config=metrics_computation_config,
                              custom_table_fields_dct=custom_table_fields_dct,
                              dataset_name='ACSPublicCoverageDataset',
                              inprocessor_name='ExponentiatedGradientReduction',
                              verbose=True)

2024-01-09 20:32:13 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 200,
 'experiment_iteration': 'Exp_iter_2',
 'fair_intervention_params_lst': "['DemographicParity']",
 'model_init_seed': 200,
 'session_uuid': 'dffb2122-b870-40d5-92b2-f0f6bfb730f0'}




Multiple alphas:   0%|          | 0/1 [00:00<?, ?it/s]

2024-01-09 20:32:13 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed


intervention_option:  DemographicParity
cur_base_flow_dataset.X_train_val.columns:  Index(['cat__SCHL_1', 'cat__SCHL_10', 'cat__SCHL_11', 'cat__SCHL_12',
       'cat__SCHL_13', 'cat__SCHL_14', 'cat__SCHL_15', 'cat__SCHL_16',
       'cat__SCHL_17', 'cat__SCHL_18', 'cat__SCHL_19', 'cat__SCHL_2',
       'cat__SCHL_20', 'cat__SCHL_21', 'cat__SCHL_22', 'cat__SCHL_23',
       'cat__SCHL_24', 'cat__SCHL_3', 'cat__SCHL_4', 'cat__SCHL_5',
       'cat__SCHL_6', 'cat__SCHL_7', 'cat__SCHL_8', 'cat__SCHL_9',
       'cat__MAR_1', 'cat__MAR_2', 'cat__MAR_3', 'cat__MAR_4', 'cat__MAR_5',
       'cat__DIS_1', 'cat__DIS_2', 'cat__ESP_0', 'cat__ESP_1', 'cat__ESP_2',
       'cat__ESP_3', 'cat__ESP_4', 'cat__ESP_5', 'cat__ESP_6', 'cat__ESP_7',
       'cat__ESP_8', 'cat__CIT_1', 'cat__CIT_2', 'cat__CIT_3', 'cat__CIT_4',
       'cat__CIT_5', 'cat__MIG_1', 'cat__MIG_2', 'cat__MIG_3', 'cat__MIL_0',
       'cat__MIL_1', 'cat__MIL_2', 'cat__MIL_3', 'cat__MIL_4', 'cat__ANC_1',
       'cat__ANC_2', 'cat__ANC_3', 'c

Analyze multiple models:   0%|          | 0/1 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

### Experiment iteration 3

In [11]:
# Configs for an experiment iteration
exp_iter_num = 3
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'
exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage

In [12]:
run_exp_iter_with_inprocessor(data_loader=exp_iter_data_loader,
                              experiment_seed=experiment_seed,
                              test_set_fraction=TEST_SET_FRACTION,
                              db_writer_func=db_writer_func,
                              fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                              metrics_computation_config=metrics_computation_config,
                              custom_table_fields_dct=custom_table_fields_dct,
                              dataset_name='ACSPublicCoverageDataset',
                              inprocessor_name='ExponentiatedGradientReduction',
                              verbose=True)

2024-01-04 19:29:01 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 300,
 'experiment_iteration': 'Exp_iter_3',
 'fair_intervention_params_lst': '[True]',
 'model_init_seed': 300,
 'session_uuid': '0626d80f-e288-4f16-b8ab-260eb34d62d3'}


Multiple alphas:   0%|          | 0/1 [00:00<?, ?it/s]

2024-01-04 19:29:01 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed
2024-01-04 19:29:01 experiment_interface.py INFO    : Models config is loaded from the input file
INFO:root:Models config is loaded from the input file


intervention_option:  True
Using ROC postprocessor
cur_base_flow_dataset.X_train_val.columns:  Index(['cat__school_GP', 'cat__school_MS', 'cat__address_R', 'cat__address_U',
       'cat__famsize_GT3', 'cat__famsize_LE3', 'cat__Pstatus_A',
       'cat__Pstatus_T', 'cat__Mjob_at_home', 'cat__Mjob_health',
       'cat__Mjob_other', 'cat__Mjob_services', 'cat__Mjob_teacher',
       'cat__Fjob_at_home', 'cat__Fjob_health', 'cat__Fjob_other',
       'cat__Fjob_services', 'cat__Fjob_teacher', 'cat__reason_course',
       'cat__reason_home', 'cat__reason_other', 'cat__reason_reputation',
       'cat__guardian_father', 'cat__guardian_mother', 'cat__guardian_other',
       'cat__schoolsup_no', 'cat__schoolsup_yes', 'cat__famsup_no',
       'cat__famsup_yes', 'cat__paid_no', 'cat__paid_yes',
       'cat__activities_no', 'cat__activities_yes', 'cat__nursery_no',
       'cat__nursery_yes', 'cat__higher_no', 'cat__higher_yes',
       'cat__internet_no', 'cat__internet_yes', 'cat__romantic_no',
     

...

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

Enabled a postprocessing mode


Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

### Experiment iteration 4

In [13]:
# Configs for an experiment iteration
exp_iter_num = 4
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'
exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage

In [14]:
run_exp_iter_with_inprocessor(data_loader=exp_iter_data_loader,
                              experiment_seed=experiment_seed,
                              test_set_fraction=TEST_SET_FRACTION,
                              db_writer_func=db_writer_func,
                              fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                              metrics_computation_config=metrics_computation_config,
                              custom_table_fields_dct=custom_table_fields_dct,
                              dataset_name='ACSPublicCoverageDataset',
                              inprocessor_name='ExponentiatedGradientReduction',
                              verbose=True)

2024-01-04 20:15:17 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 400,
 'experiment_iteration': 'Exp_iter_4',
 'fair_intervention_params_lst': '[True]',
 'intervention_param': 'True',
 'model_init_seed': 400,
 'session_uuid': '0626d80f-e288-4f16-b8ab-260eb34d62d3'}


Multiple alphas:   0%|          | 0/1 [00:00<?, ?it/s]

2024-01-04 20:15:17 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed
2024-01-04 20:15:17 experiment_interface.py INFO    : Models config is loaded from the input file
INFO:root:Models config is loaded from the input file


intervention_option:  True
Using ROC postprocessor
cur_base_flow_dataset.X_train_val.columns:  Index(['cat__school_GP', 'cat__school_MS', 'cat__address_R', 'cat__address_U',
       'cat__famsize_GT3', 'cat__famsize_LE3', 'cat__Pstatus_A',
       'cat__Pstatus_T', 'cat__Mjob_at_home', 'cat__Mjob_health',
       'cat__Mjob_other', 'cat__Mjob_services', 'cat__Mjob_teacher',
       'cat__Fjob_at_home', 'cat__Fjob_health', 'cat__Fjob_other',
       'cat__Fjob_services', 'cat__Fjob_teacher', 'cat__reason_course',
       'cat__reason_home', 'cat__reason_other', 'cat__reason_reputation',
       'cat__guardian_father', 'cat__guardian_mother', 'cat__guardian_other',
       'cat__schoolsup_no', 'cat__schoolsup_yes', 'cat__famsup_no',
       'cat__famsup_yes', 'cat__paid_no', 'cat__paid_yes',
       'cat__activities_no', 'cat__activities_yes', 'cat__nursery_no',
       'cat__nursery_yes', 'cat__higher_no', 'cat__higher_yes',
       'cat__internet_no', 'cat__internet_yes', 'cat__romantic_no',
     

...

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

Enabled a postprocessing mode


Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

### Experiment iteration 5

In [11]:
# Configs for an experiment iteration
exp_iter_num = 5
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'
exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage

In [12]:
run_exp_iter_with_inprocessor(data_loader=exp_iter_data_loader,
                              experiment_seed=experiment_seed,
                              test_set_fraction=TEST_SET_FRACTION,
                              db_writer_func=db_writer_func,
                              fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                              metrics_computation_config=metrics_computation_config,
                              custom_table_fields_dct=custom_table_fields_dct,
                              dataset_name='ACSPublicCoverageDataset',
                              inprocessor_name='ExponentiatedGradientReduction',
                              verbose=True)

2024-01-05 03:55:49 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 500,
 'experiment_iteration': 'Exp_iter_5',
 'fair_intervention_params_lst': '[True]',
 'model_init_seed': 500,
 'session_uuid': '0626d80f-e288-4f16-b8ab-260eb34d62d3'}


Multiple alphas:   0%|          | 0/1 [00:00<?, ?it/s]

2024-01-05 03:55:49 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed
2024-01-05 03:55:49 experiment_interface.py INFO    : Models config is loaded from the input file
INFO:root:Models config is loaded from the input file


intervention_option:  True
Using ROC postprocessor
cur_base_flow_dataset.X_train_val.columns:  Index(['cat__school_GP', 'cat__school_MS', 'cat__address_R', 'cat__address_U',
       'cat__famsize_GT3', 'cat__famsize_LE3', 'cat__Pstatus_A',
       'cat__Pstatus_T', 'cat__Mjob_at_home', 'cat__Mjob_health',
       'cat__Mjob_other', 'cat__Mjob_services', 'cat__Mjob_teacher',
       'cat__Fjob_at_home', 'cat__Fjob_health', 'cat__Fjob_other',
       'cat__Fjob_services', 'cat__Fjob_teacher', 'cat__reason_course',
       'cat__reason_home', 'cat__reason_other', 'cat__reason_reputation',
       'cat__guardian_father', 'cat__guardian_mother', 'cat__guardian_other',
       'cat__schoolsup_no', 'cat__schoolsup_yes', 'cat__famsup_no',
       'cat__famsup_yes', 'cat__paid_no', 'cat__paid_yes',
       'cat__activities_no', 'cat__activities_yes', 'cat__nursery_no',
       'cat__nursery_yes', 'cat__higher_no', 'cat__higher_yes',
       'cat__internet_no', 'cat__internet_yes', 'cat__romantic_no',
     

...

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

Enabled a postprocessing mode


Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

### Experiment iteration 6

In [13]:
# Configs for an experiment iteration
exp_iter_num = 6
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'
exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage

In [14]:
run_exp_iter_with_inprocessor(data_loader=exp_iter_data_loader,
                              experiment_seed=experiment_seed,
                              test_set_fraction=TEST_SET_FRACTION,
                              db_writer_func=db_writer_func,
                              fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                              metrics_computation_config=metrics_computation_config,
                              custom_table_fields_dct=custom_table_fields_dct,
                              dataset_name='ACSPublicCoverageDataset',
                              inprocessor_name='ExponentiatedGradientReduction',
                              verbose=True)

2024-01-05 04:49:43 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 600,
 'experiment_iteration': 'Exp_iter_6',
 'fair_intervention_params_lst': '[True]',
 'intervention_param': 'True',
 'model_init_seed': 600,
 'session_uuid': '0626d80f-e288-4f16-b8ab-260eb34d62d3'}


Multiple alphas:   0%|          | 0/1 [00:00<?, ?it/s]

2024-01-05 04:49:43 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed
2024-01-05 04:49:43 experiment_interface.py INFO    : Models config is loaded from the input file
INFO:root:Models config is loaded from the input file


intervention_option:  True
Using ROC postprocessor
cur_base_flow_dataset.X_train_val.columns:  Index(['cat__school_GP', 'cat__school_MS', 'cat__address_R', 'cat__address_U',
       'cat__famsize_GT3', 'cat__famsize_LE3', 'cat__Pstatus_A',
       'cat__Pstatus_T', 'cat__Mjob_at_home', 'cat__Mjob_health',
       'cat__Mjob_other', 'cat__Mjob_services', 'cat__Mjob_teacher',
       'cat__Fjob_at_home', 'cat__Fjob_health', 'cat__Fjob_other',
       'cat__Fjob_services', 'cat__Fjob_teacher', 'cat__reason_course',
       'cat__reason_home', 'cat__reason_other', 'cat__reason_reputation',
       'cat__guardian_father', 'cat__guardian_mother', 'cat__guardian_other',
       'cat__schoolsup_no', 'cat__schoolsup_yes', 'cat__famsup_no',
       'cat__famsup_yes', 'cat__paid_no', 'cat__paid_yes',
       'cat__activities_no', 'cat__activities_yes', 'cat__nursery_no',
       'cat__nursery_yes', 'cat__higher_no', 'cat__higher_yes',
       'cat__internet_no', 'cat__internet_yes', 'cat__romantic_no',
     

...

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGBM] [Warning] min_data_in_leaf is set=100, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=100
[LightGB

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

Enabled a postprocessing mode


Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]